In [ ]:
%matplotlib qt

In [ ]:
# MEG files
%ll camcan/BIDSsep/passive

In [ ]:
# emptyroom files (no patient during the recording)
%ll camcan/emptyroom/

In [ ]:
# transformation files (head-to-MRI)
%ll camcan/trans/

In [ ]:
# MRI
%ll camcan/freesurfer/

In [ ]:
# CALIBRATION file
%ll sss_cal.dat

In [ ]:
# CROSSTALK file
%ll ct_sparse.fif

# Load libraries and define paths

In [ ]:
from autoreject import get_rejection_threshold
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import mne
from nilearn.plotting import plot_stat_map

PLOT = True
PLOT_3D = PLOT and False

# CC720986: good visual ERP tomo
SUBJECT = 'CC723395'  # CC220352, CC720986, CC721519
# DATA_PATH = '/storage/store/data/camcan/BIDSsep'
DATA_PATH = 'camcan/BIDSsep'
EMPTYROOM_PATH = 'camcan/emptyroom'
FREESURFER_PATH = 'camcan/freesurfer'
TRANS_PATH = 'camcan/trans'
CALIBRATION_PATH = 'sss_cal.dat'
CROSSTALK_PATH = 'ct_sparse.fif'

# Load raw data and empty room recordings

In [ ]:
data_folder = os.path.join(DATA_PATH, 'passive', 'sub-' + SUBJECT, 'ses-passive')
filename = 'sub-' + SUBJECT + '_ses-passive_task-passive_meg.fif'
data_raw_file = os.path.join(data_folder, 'meg', filename)
raw = mne.io.read_raw_fif(data_raw_file, preload=True, verbose=False)

data_er_folder = os.path.join(EMPTYROOM_PATH, SUBJECT)
filename = 'emptyroom_' + SUBJECT + '.fif'
data_raw_er_file = os.path.join(data_er_folder, filename)
raw_er = mne.io.read_raw_fif(data_raw_er_file, preload=True, verbose=False)

# Plot raw data and psd

In [ ]:
if PLOT:
    raw.copy().pick(['meg']).plot(duration=1, start=40, scalings=2*1e-10, n_channels=5)
    plt.show()

In [ ]:
if PLOT:
    raw.compute_psd().plot()
    plt.show()

# Find bad channels and maxfilter

In [ ]:
raw_check = raw.copy()
auto_noisy_chs, auto_flat_chs, auto_scores = mne.preprocessing.find_bad_channels_maxwell(
    raw.copy(), cross_talk=CROSSTALK_PATH, calibration=CALIBRATION_PATH,
    return_scores=True, verbose=False)
raw.info['bads'] = auto_noisy_chs + auto_flat_chs
raw_er.info['bads'] = raw.info['bads']

In [ ]:
raw.info['bads']

In [ ]:
raw_sss = mne.preprocessing.maxwell_filter(
    raw.copy(), cross_talk=CROSSTALK_PATH, calibration=CALIBRATION_PATH, verbose=False)
raw_er_sss = mne.preprocessing.maxwell_filter(
    raw_er.copy(), cross_talk=CROSSTALK_PATH, calibration=CALIBRATION_PATH, coord_frame='meg', verbose=False)

In [ ]:
# plt comparison
if PLOT:
    raw.copy().pick(['meg']).plot(duration=1.0, start=40.0, scalings=1e-9, n_channels=5)
    raw_sss.copy().pick(['meg']).plot(duration=1.0, start=40.0, scalings=1e-9, n_channels=5)
    raw.compute_psd().plot()
    raw_sss.compute_psd().plot()
    plt.show()

# Low pass filter

In [ ]:
raw_sss.filter(l_freq=1, h_freq=30, verbose=False)
raw_er_sss.filter(l_freq=1, h_freq=30, verbose=False)
# raw_sss.notch_filter(np.arange(50, 201, 50))
raw_sss = raw_sss.crop(tmax=130)
if PLOT:
    raw_sss.copy().pick(['meg']).plot(duration=150.0, start=0.0, scalings=5*1e-11, n_channels=5)
    raw_sss.compute_psd().plot()
    plt.show()

# Load events, create epochs and evoked

In [ ]:
event_dict = {'auditory/300Hz': 6, 'auditory/600Hz': 7, 'auditory/1200Hz': 8, 'visual': 9}
events = mne.find_events(raw_sss, verbose=False)
if PLOT:
    fig = mne.viz.plot_events(
        events, event_id=event_dict, sfreq=raw_sss.info['sfreq'], first_samp=raw_sss.first_samp)
    plt.show()

In [ ]:
# create epochs
TMIN, TMAX = -0.2, 0.5
epochs = mne.Epochs(
    raw_sss, events, tmin=TMIN, tmax=TMAX, baseline=(TMIN, 0.0), event_id=event_dict, preload=True, verbose=False)
epochs.get_data().shape

In [ ]:
epochs.plot()

In [ ]:
# reject some epochs
reject = get_rejection_threshold(epochs, verbose=False)
print(reject)
epochs = mne.Epochs(
    raw_sss, events, tmin=TMIN, tmax=TMAX, baseline=(TMIN, 0.0), reject=reject,
    event_id=event_dict, preload=True, verbose=False)
epochs.get_data().shape

In [ ]:
evoked = dict()
for event in event_dict:
    evoked[event] = epochs[event].average()

In [ ]:
EVENT = 'visual'
# EVENT = 'auditory'
if PLOT:
    print(EVENT)
    epochs[EVENT].average().plot()
    plt.show()

In [ ]:
all_times = np.linspace(0.1, 0.3, num=6)
if PLOT:
    evoked[EVENT].plot_topomap(all_times, ch_type='mag')
    plt.show()

# Compute noise covariance from baseline segments

In [ ]:
noise_cov = mne.compute_covariance(epochs, tmax=0, method='auto', rank=None, verbose=False)
if PLOT:
    noise_cov.plot(raw_sss.info)
    plt.show()

In [ ]:
if PLOT:
    print(EVENT)
    evoked[EVENT].plot_white(noise_cov, time_unit='s', verbose=False)
    plt.show()

# Compute noise covariance from empty room recordings

In [ ]:
if PLOT:
    raw_er_sss.copy().pick(['meg']).plot(duration=150.0, start=0.0, scalings=5*1e-11, n_channels=5)
    plt.show()

In [ ]:
noise_cov_er = mne.compute_raw_covariance(raw_er_sss, tmin=0, tmax=None, verbose=False)
if PLOT:
    print('Noise covariance matrix:')
    noise_cov.plot(raw_er.info, show_svd=False)
    print('Noise covariance matrix empty room:')
    noise_cov_er.plot(raw_er_sss.info, show_svd=False)
    plt.show()

In [ ]:
if PLOT:
    print(EVENT)
    evoked[EVENT].plot_white(noise_cov_er, time_unit='s', verbose=False)
    plt.show()

# Compute the dSPM inverse solution on the cortical surface 

## Plot the coregistration

The coregistration is the operation that allows to position the head and the sensors in a common coordinate system.

In [ ]:
trans_file = os.path.join(TRANS_PATH, 'sub-' + SUBJECT + '-trans.fif')
if PLOT_3D:
    mne.viz.plot_alignment(
        raw_sss.info, trans_file, subject=SUBJECT, dig=True,
        meg=['helmet', 'sensors'], subjects_dir=FREESURFER_PATH, surfaces='head-dense', verbose=False)

## Plot BEM and source space

bem = boundary element model

The BEM surfaces are the triangulations of the interfaces between different tissues needed for forward computation. These surfaces are for example the inner skull surface, the outer skull surface and the outer skin surface, a.k.a. scalp surface.

In [ ]:
# src = mne.setup_source_space(SUBJECT, spacing='oct4', subjects_dir=FREESURFER_PATH, verbose=False)

SPHERE = np.array([0, -0.005, 0, 0.085])
surface_path = os.path.join(FREESURFER_PATH, SUBJECT, 'bem', 'inner_skull.surf')
src = mne.setup_volume_source_space(
    SUBJECT, surface=surface_path, subjects_dir=FREESURFER_PATH, sphere=SPHERE, verbose=False)

In [ ]:
if PLOT:
    mne.viz.plot_bem(SUBJECT, subjects_dir=FREESURFER_PATH, src=src)
    plt.show()

## Plot sources in 3D

In [ ]:
PLOT_3D = True
if PLOT_3D:
    fig = mne.viz.plot_alignment(info=raw.info, trans=trans_file,
        subject=SUBJECT, subjects_dir=FREESURFER_PATH, surfaces='white', coord_frame='mri', src=src)
    mne.viz.set_3d_view(fig)

## Make bem model

In [ ]:
# CONDUCTIVITY = (0.3, 0.006, 0.3)  # for three layers
CONDUCTIVITY = (0.3,)  # for single layer
model = mne.make_bem_model(SUBJECT, ico=4, conductivity=CONDUCTIVITY, subjects_dir=FREESURFER_PATH, verbose=False)
bem = mne.make_bem_solution(model, verbose=False)

In [ ]:
fwd = mne.make_forward_solution(data_raw_file, trans=trans_file, src=src, bem=bem, verbose=False)
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

In [ ]:
inverse_operator = mne.minimum_norm.make_inverse_operator(raw_sss.info, fwd, noise_cov, verbose=False)

In [ ]:
METHOD = 'dSPM'
print(EVENT)
stc = mne.minimum_norm.apply_inverse(evoked[EVENT], inverse_operator, method=METHOD, verbose=False)
if PLOT:
    stc.plot(src=src, subjects_dir=FREESURFER_PATH)

# Electroocoulogram

In [ ]:
eog_evoked = mne.preprocessing.create_eog_epochs(raw_sss, verbose=False).average()
eog_evoked.apply_baseline(baseline=(None, -0.2))  # subtract mean signal
if PLOT:
    eog_evoked.plot()
    plt.show()

# Electrocardiogram

In [ ]:
ecg_evoked = mne.preprocessing.create_ecg_epochs(raw_sss, verbose=False).average()
ecg_evoked.apply_baseline(baseline=(None, -0.2))
ecg_evoked.plot()
plt.show()

# ICA

In [ ]:
ica = mne.preprocessing.ICA(n_components=0.999, method='picard', max_iter='auto', random_state=123)
ica.fit(raw_sss)

In [ ]:
if PLOT:
    ica.plot_components()
    plt.show()

In [ ]:
BLINK_CHANNELS = [1]
HEARTBEAT_CHANNELS = [7, 14]
if PLOT:
    ica.plot_sources(raw_sss, picks=BLINK_CHANNELS+HEARTBEAT_CHANNELS)
    plt.show()

In [ ]:
if PLOT:
    # blinks
    ica.plot_overlay(raw_sss, exclude=BLINK_CHANNELS, picks='mag')
    # heartbeats
    ica.plot_overlay(raw_sss, exclude=HEARTBEAT_CHANNELS, picks='mag')
    plt.show()

In [ ]:
ica.exclude = BLINK_CHANNELS+HEARTBEAT_CHANNELS
reconst_raw_sss = raw_sss.copy()
ica.apply(reconst_raw_sss)

In [ ]:
epochs = mne.Epochs(
    raw_sss, events, tmin=TMIN, tmax=TMAX, baseline=(TMIN, 0.0), reject=reject,
    event_id=event_dict, preload=True, verbose=False)
evoked = dict()
for event in event_dict:
    evoked[event] = epochs[event].average()
if PLOT:
    print(EVENT)
    evoked[EVENT].plot()
    plt.show()

epochs = mne.Epochs(
    reconst_raw_sss, events, tmin=TMIN, tmax=TMAX, baseline=(TMIN, 0.0), reject=reject,
    event_id=event_dict, preload=True, verbose=False)
evoked = dict()
for event in event_dict:
    evoked[event] = epochs[event].average()
if PLOT:
    print(EVENT)
    evoked[EVENT].plot()
    plt.show()
all_times = np.linspace(0.1, 0.3, num=6)
if PLOT:
    evoked[EVENT].plot_topomap(all_times, ch_type='mag')
    plt.show()